# Opt Art Chapter 2: Truchet Tiles

### Libraries to import

In [1]:
import sys
import numpy as np
import PIL 
from PIL import Image, ImageDraw, ImageFilter, ImageStat
import math
from math import gcd

### Background Functions (must run first)

In [2]:
def chapter_2(file, scale, pattern,save,**kwargs):
    pixel_size  = kwargs.get('pixel_size', 5)
    ##import image and make it grayscale
    import_im = Image.open(file, mode='r', formats=None).convert('L')
    ##resize image based on scale
    resized_im = import_im.resize((import_im._size[0]*scale,import_im._size[1]*scale))
    ##create blank image of same size to put pattern on
    blank_im = Image.new('L', (resized_im._size[0],resized_im._size[1]), 128)
    ##creating drawing function for blank image
    draw = ImageDraw.Draw(blank_im)
    
    ##deteremine how many pixels the tiles will cover when recreating the image
    new_pixels, new_pixel_size = tile_pixels([],import_im,resized_im, pattern,pixel_size)
    tile_image(new_pixels, new_pixel_size, resized_im, pattern, draw, blank_im,save)
    

In [3]:
def tile_pixels(t_list,orig_image,re_image, pattern,pixel_size):
    ##determine how to partition your image for tiling
    new_pixel_size = determine_pixel_size(orig_image._size[0], orig_image._size[1], scale,pixel_size)

    ##get t values depending on pattern choice
    if pattern == 'truchet':
        for r in range(0,re_image._size[0], new_pixel_size):
            for c in range(0,re_image._size[1],new_pixel_size):
                square = re_image.crop((r,c,r+new_pixel_size,c+new_pixel_size))
                stat = ImageStat.Stat(square)
                average = (int(stat.mean[0])/255)
            
                t_list.append(get_t(average))
                
    elif pattern == 'star':    
        for r in range(0,re_image._size[0], new_pixel_size):
            for c in range(0,re_image._size[1],new_pixel_size):
                square = re_image.crop((r,c,r+new_pixel_size,c+new_pixel_size))
                stat = ImageStat.Stat(square)
                average = (int(stat.mean[0])/255)
            
                t_list.append(get_t_star(average))
    else:
        raise Exception("Not a valid pattern input")

    ##convert t values to array which can be used for tiling purposes
    t_vals_array = np.array(t_list)
    t_vals_array.resize((int(re_image._size[0]/new_pixel_size),int(re_image._size[1]/new_pixel_size)))
    return (t_vals_array, new_pixel_size)

In [4]:
def cf(num1,num2):
    ###create list of factors for two numbers
    n=[]
    g=gcd(num1, num2)
    for i in range(1, g+1): 
        if g%i==0: 
            n.append(i)
    return n

In [5]:
def closest_num(myList,myNumber):
    ###Find closest number in a list to the input number
    return min(myList, key=lambda x:abs(x-myNumber))

In [6]:
def determine_pixel_size(im_length, im_width, scale,pix_sz):
    ##deteremine an n by n square of pixels to make into tile 
    ###check if pixel size input is too big for image
    if (pix_sz > im_length) or (pix_sz > im_width):
         raise Exception("Too big for pixel size")
    else:
        ###create list of factors for length and width of image
        fact_list = cf(im_width,im_length)
        ###find factor in list that is closest to the input pixel size
        pixel_size = closest_num(fact_list,pix_sz)
        ###scale new pixel size properly for image
        pixel_size = pixel_size*scale
    
    return pixel_size

In [7]:
def tile_image(new_pix, pix_size, image, pattern, draw, new_im,save):
    ##tile image based on truchet tile pattern
    if pattern == 'truchet':
        for r in range(0,image._size[0],pix_size):
            for c in range(0,image._size[1],pix_size):
                t = new_pix[int(r/pix_size)][int(c/pix_size)]
                ###creates truchet tile based on {a,c},{b,d} pattern and t value for each tile
                if (r%(2*pix_size)==0 and c%(2*pix_size)==0):
                    square_a = ((0, 0), (0,1), (1, 1), (1,0))
                    triangle_a = ((0, 0), (1,0), (1, 1),((1-t),t))
                    square_a = tuple([tile_trans(x,right_shift = r, down_shift= c, scale = pix_size) for x in square_a])
                    triangle_a = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in triangle_a])
                    draw.polygon(square_a,fill = 0)
                    draw.polygon(triangle_a,fill= 255)
            
                elif(r%(2*pix_size)==pix_size and c%(2*pix_size)==0):
                    square_b = ((0, 0), (0,1), (1, 1), (1,0))
                    triangle_b = ((0, 1), (1,1), (1, 0),(1-t,1-t))
                    square_b = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in square_b])
                    triangle_b = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in triangle_b])
                    draw.polygon(square_b,fill = 0)
                    draw.polygon(triangle_b,fill= 255)
            
                elif(r%(2*pix_size)==0 and c%(2*pix_size)==pix_size):
                    square_d = ((0, 0), (0,1), (1, 1), (1,0))
                    triangle_d = ((0, 1), (0,0), (1, 0),(t,t))
                    square_d = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in square_d])
                    triangle_d = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in triangle_d])
                    draw.polygon(square_d,fill = 0)
                    draw.polygon(triangle_d,fill= 255)
            
                elif(r%(2*pix_size)==pix_size and c%(2*pix_size)==pix_size):
                    square_c = ((0, 0), (0,1), (1, 1), (1,0))
                    triangle_c = ((0, 0), (0,1), (1, 1),(t,(-t+1)))
                    square_c = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in square_c])
                    triangle_c = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in triangle_c])
                    draw.polygon(square_c,fill = 0)
                    draw.polygon(triangle_c,fill= 255)
    ##tile image based on star pattern
    elif pattern == 'star':
        for r in range(0,image._size[0], pix_size):
            for c in range(0,image._size[1],pix_size):
                t = new_pix[int(r/pix_size)][int(c/pix_size)]
                
                ###creates star based on t values for that tile
                square = ((0, 0), (0,1), (1, 1), (1,0))
                triangle_top = ((0, 0), (1,0), (0.5, t))
                triangle_right = ((1, 0), (1,1), (1-t, 0.5))
                triangle_bottom = ((1, 1), (0,1), (0.5, 1-t))
                triangle_left = ((0, 1), (0,0), (t, 0.5))
                square = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in square])
                triangle_top = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in triangle_top])
                triangle_right = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in triangle_right])
                triangle_bottom = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in triangle_bottom])
                triangle_left = tuple([tile_trans(x,right_shift = r, down_shift= c,scale = pix_size) for x in triangle_left])

                draw.polygon(square,fill = 255)
                draw.polygon(triangle_top,fill= 0)
                draw.polygon(triangle_right,fill= 0)
                draw.polygon(triangle_bottom,fill= 0)
                draw.polygon(triangle_left,fill= 0)
    
    else:
        ###error if invalid pattern put into function
        raise Exception("Not a valid pattern input")
    ##show image once tiling is completed
    new_im.show()
    
    ##saves image as 'tiled_image.jpg'
    if save == True:
        image.save('tiled_image.jpg')

In [8]:
# 0.25+0.5t = Beta_ij
#convert brightness value to t value for truchet tile
def get_t(beta_ij):
    if beta_ij < 0.25:
        t = 0.25
    elif beta_ij > 0.75:
        t = 0.75
    else: 
        t = 0.5*beta_ij+0.25
    return t

In [9]:
# t = -0.4beta_ij+0.5
#convert brightness value to t value for star tile
def get_t_star(beta_ij):
    if beta_ij < 0.25:
        t = 0.4
    elif beta_ij > 0.75:
        t = 0.2
    else: 
        t = -0.4*beta_ij+0.5
    return t

In [10]:
def tile_trans(point,**kwargs):
    ##apply transformation to tile to move into correct spot in pattern/image
    scale = kwargs.get('scale',1)
    right_shift = kwargs.get('right_shift',0)
    down_shift = kwargs.get('down_shift',0)
    updated_point = point
    
    ####scale points
    if scale != 1:
        updated_point = tuple([scale*x for x in updated_point])
    
    ####shift right or down
    if right_shift != 0 and down_shift !=0:
        updated_point = (updated_point[0] + right_shift, updated_point[1]+down_shift)
    elif right_shift != 0:
        updated_point = (updated_point[0]+right_shift, updated_point[1])
    elif down_shift != 0:
        updated_point = (updated_point[0], updated_point[1]+down_shift)
        
    return updated_point

## Try it Yourself!

#### Adjust Inputs Here

In [13]:
### input the file name for the image you would like to be processed 
###(note that file must be in the same folder as the jupyter notebook to work)
file ='duck.jpg'
###adjust scale if your image is too small (cannot be less than 1)
scale = 10
####indicate which pattern (truchet or star) you would like for your image
pattern = 'star'
####indicate if you would like to save your masterpiece (True) or not (False)
save_image = False

#### Run the cell below once you are satisifed with your inputs

In [14]:
chapter_2(file, scale, pattern,save_image, pixel_size = 5)